In [1]:
import pandas as pd
import httpx

In [ ]:
## SENNET API is A MESS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import requests
url = 'https://search.api.sennetconsortium.org/param-search/datasets?dataset_type=Histology'
#url = 'https://search.api.sennetconsortium.org/param-search/sources?group_name=TMC%20-%20Johns%20Hopkins%20University'

response = requests.get(url)
response

<Response [200]>

In [19]:
url

'https://search.api.sennetconsortium.org/param-search/datasets?dataset_type=Histology'

In [ ]:
dataset_type = "Histology"
organ = "UBERON_0001013" # SP 
base_url = "https://search.api.sennetconsortium.org/param-search/datasets"
params = {
        "dataset_type": dataset_type,
        #"organ": organ
    }

#url = 'https://search.api.sennetconsortium.org/param-search/sources?group_name=TMC%20-%20Johns%20Hopkins%20University'
with httpx.Client() as client:
    response = client.get(base_url, params=params)

response.json()

/tmp/ipykernel_186788/1475976656.py:2: DtypeWarning: Columns (2,3,4,5,6,7,10,12,17,29,31,32,33,44,48,51,52,54,57,58,59,62,64,65,75,79,83,85,86,88,90,94,99,101,103,105,106,109,111,113,116,117,118,120,121,123,127,128,129,130,132,134,139,140,141,142,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,166,170,172,174,184,191,194,196,198,199,201,204,208,218,220,225,226,228,229,230,232,234,240,242,243,245,247,248) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tem = pd.read_csv(Path_csv, sep='\t')


'uuid,hubmap_id,ablation_distance_between_shots_x_units,ablation_distance_between_shots_x_value,ablation_distance_between_shots_y_units,ablation_distance_between_shots_y_value,ablation_frequency_unit,ablation_frequency_value,acquisition_instrument_model,acquisition_instrument_vendor,activation_stimulus,amount_of_input_analyte_unit,amount_of_input_analyte_value,analysis_protocol_doi,analyte_class,antibody_reagent_kit,area_normalized_ion_dose_unit,area_normalized_ion_dose_value,assay_category,assay_input_entity,assay_type,barcode_offset,barcode_read,barcode_size,bead_barcode_offset,bead_barcode_read,bead_barcode_size,bulk_atac_cell_isolation_protocols_io_doi,bulk_rna_isolation_protocols_io_doi,bulk_rna_isolation_quality_metric_value,bulk_rna_yield_units_per_tissue_unit,bulk_rna_yield_value,bulk_transposition_input_number_nuclei,calibration_kit_lot_number,capture_area_id,capture_batch_id,cell_barcode_offset,cell_barcode_read,cell_barcode_size,cell_boundary_marker_or_stain,created_by_user_

array(['This is the entity from which the analyte is being captured. For example, for bulk sequencing this would be "tissue", while it would be "single cell" for single cell sequencing. This field is used to determine which analysis pipeline to run.',
       nan, 'single nucleus', 'tissue (bulk)', 'spot', 'single cell'],
      dtype=object)

In [35]:
def filter_by_organ(df, organ="heart"):
    return df[df['origin_samples_unique_mapped_organs'].str.contains(organ, case=False, na=False)]


def build_dataset_descriptions(df):
    """
    Build a list of dicts with HuBMAP dataset URLs and descriptions from selected columns.
    
    Args:
        df (pd.DataFrame): A DataFrame with columns including:
            - 'uuid', 'donor.hubmap_id', 'puck_id', 'pi',
              'origin_samples_unique_mapped_organs', 'analyte_class',
              'assay_type', 'assay_input_entity', 'spatial_target',
              'sc_isolation_entity', 'sc_isolation_tissue_dissociation'
    
    Returns:
        List[dict]: Each dict has keys 'url' and 'description'
    """
    results = []

    for _, row in df.iterrows():
        uuid = row.get('uuid', '')
        url = f"https://portal.hubmapconsortium.org/browse/dataset/{uuid}"

        description_fields = [
            row.get('pi', ''),
            row.get('origin_samples_unique_mapped_organs', ''),
            row.get('analyte_class', ''),
            row.get('assay_type', ''),
            row.get('assay_input_entity', ''),
            row.get('spatial_target', ''),
            row.get('sc_isolation_entity', ''),
            row.get('sc_isolation_tissue_dissociation', ''),
        ]
        # Filter out None or empty string, then join with " | "
        description = " | ".join(str(f) for f in description_fields if pd.notna(f) and str(f).strip())
        results.append({
            'url': url,
            'description': description
        })

    return results

# Generate result
heart_df = build_dataset_descriptions(filter_by_organ(df_tem, "heart"))
heart_df

[{'url': 'https://portal.hubmapconsortium.org/browse/dataset/ba41e71358136f6a202114681a217a95',
  'description': "['Heart']"},
 {'url': 'https://portal.hubmapconsortium.org/browse/dataset/7e4c58d746cdd5b36503a12c38974b0a',
  'description': "Cole Trapnell | ['Heart'] | DNA | sciATACseq | nucleus | TRUE"},
 {'url': 'https://portal.hubmapconsortium.org/browse/dataset/d180976b42a9ed8b5fb12a508f79a238',
  'description': "['Heart']"},
 {'url': 'https://portal.hubmapconsortium.org/browse/dataset/21043afd6be7ec258f8a9fa1577c41b5',
  'description': "['Heart']"},
 {'url': 'https://portal.hubmapconsortium.org/browse/dataset/c616de3ce57804ede2dd519a1c8addf2',
  'description': "['Heart']"},
 {'url': 'https://portal.hubmapconsortium.org/browse/dataset/acaf75b8292db4a79dc14e3021742217',
  'description': "Cole Trapnell | ['Heart'] | DNA | sciATACseq | nucleus | xyz"},
 {'url': 'https://portal.hubmapconsortium.org/browse/dataset/2d3dbd5e5a3d9d5cfc7a46f06815a4eb',
  'description': "['Heart']"},
 {'url':

In [ ]:
from os import environ
from hubmap_api_py_client import Client
#client = Client(environ['API_ENDPOINT'])
endpoint = environ.get('API_ENDPOINT', 'https://cells.api.hubmapconsortium.org/api/')#'https://search.api.hubmapconsortium.org/v3/')
#'https://cells.api.hubmapconsortium.org/api')#'https://default.api.endpoint/v3')
client = Client(endpoint)

In [17]:
#[m for m in dir(client) if m.startswith('select_')]
#gene_symbol = 
#client.select_genes(where="modality", has=["rna"]).get_list()[0]['gene_symbol']

In [25]:
cell_type = client.select_celltypes().get_list()
#cells_with_celltype = client.select_cells(where='celltype', has=[cell_type])
#assert len(cells_with_celltype) > 0


In [34]:
organ_name = client.select_organs().get_list()[0]['grouping_name']
cells_in_organ = client.select_cells(where='organ', has=[organ_name])
assert len(cells_in_organ) > 0


In [69]:
[m for m in dir(client) if m.startswith('select_')]
['select_cells', 'select_celltypes', 'select_clusters', 'select_datasets', 'select_genes', 'select_organs', 'select_proteins']

gene_symbol = client.select_genes(where="modality", has=["rna"]).get_list()[0]['gene_symbol']
cells_with_gene = client.select_cells(where='gene', has=[f'{gene_symbol} > 0.5'], genomic_modality='rna')
assert len(cells_with_gene) > 0

# Select cells from the datasets with the following UUIDs:
dataset_a_uuid = client.select_datasets(where="gene", has=[f'{gene_symbol} > 1'], genomic_modality="rna", min_cell_percentage=0.0).get_list()[0]['uuid']
dataset_b_uuid = client.select_datasets(where="gene", has=[f'{gene_symbol} > 1'], genomic_modality="rna", min_cell_percentage=0.0).get_list()[1]['uuid']
cells_in_a_len = len(client.select_cells(where='dataset', has=[dataset_a_uuid]))
cells_in_b_len = len(client.select_cells(where='dataset', has=[dataset_b_uuid]))
cells_in_datasets = client.select_cells(where='dataset', has=[dataset_a_uuid, dataset_b_uuid])
cells_in_datasets_len = len(cells_in_datasets)
assert cells_in_datasets_len > 0
assert cells_in_datasets_len == cells_in_a_len + cells_in_b_len

# Combine criteria with intersection:
cells_with_gene_in_datasets = cells_with_gene & cells_in_datasets

# Get a list; should run quickly:
cell_list = cells_with_gene.get_list()

cells = cell_list[0:10]
assert len(cells) > 0
assert cells[0].keys() == {'cell_id', 'modality', 'dataset', 'organ', 'cell_type' ,'clusters'}